In [2]:
!pip install pyspark==2.4.5

/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
     |████████████████████████████████| 217.8 MB 6.2 kB/s  eta 0:00:01    |█████▊                          | 39.0 MB 9.2 MB/s eta 0:00:20██████████████▎           | 138.1 MB 26.4 MB/s eta 0:00:04     |█████████████████████▍          | 145.7 MB 22.0 MB/s eta 0:00:04     |█████████████████████████████▏  | 198.7 MB 21.1 MB/s eta 0:00:01
     |████████████████████████████████| 197 kB 29.1 MB/s eta 0:00:01
  Created wheel for pyspark: filename=pyspark-2.4.5-py2.py3-none-any.whl size=218257928 sha256=3330016e086feba92030bccad84d1f7010389ec52e1e60

In [3]:
try:
    from pyspark import SparkContext, SparkConf
    from pyspark.sql import SparkSession
except ImportError as e:
    printmd('<<<<<!!!!! Please restart your kernel after installing Apache Spark !!!!!>>>>>')

In [4]:
sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))

spark = SparkSession \
    .builder \
    .getOrCreate()

# Exercise 2
## Part 1
Now let's calculate covariance and correlation by ourselves using ApacheSpark

1st we crate two random RDD’s, which shouldn't correlate at all.


In [5]:
import random
rddX = sc.parallelize(random.sample(list(range(100)),100))
rddY = sc.parallelize(random.sample(list(range(100)),100))

Now we calculate the mean, note that we explicitly cast the denominator to float in order to obtain a float instead of int

In [6]:
meanX = rddX.sum()/float(rddX.count())
meanY = rddY.sum()/float(rddY.count())
print (meanX)
print (meanY)

49.5
49.5


Now we calculate the covariance

In [8]:
rddXY = rddX.zip(rddY)
covXY = rddXY.map(lambda x_y : (x_y[0]-meanX)*(x_y[1]-meanY)).sum()/rddXY.count()
covXY

80.04

Covariance is not a normalized measure. Therefore we use it to calculate correlation. But before that we need to calculate the indivicual standard deviations first

In [9]:
from math import sqrt

n = rddXY.count()
sdX = sqrt(rddX.map(lambda x : pow(x-meanX,2)).sum()/n)
sdY = sqrt(rddY.map(lambda x : pow(x-meanY,2)).sum()/n)
print (sdX)
print (sdY)

28.86607004772212
28.86607004772212


Now we calculate the correlation

In [10]:
corrXY = covXY / (sdX * sdY)
corrXY

0.09605760576057606

## Part 2
No we want to create a correlation matrix out of the four RDDs used in the lecture

In [11]:
from pyspark.mllib.stat import Statistics
import random

column1 = sc.parallelize(range(100))
column2 = sc.parallelize(range(100,200))
column3 = sc.parallelize(list(reversed(range(100))))
column4 = sc.parallelize(random.sample(range(100),100))
data = column1.zip(column2).zip(column3).zip(column4).map(lambda a_b_c_d : (a_b_c_d[0][0][0],a_b_c_d[0][0][1],a_b_c_d[0][1],a_b_c_d[1]) ).map(lambda a_b_c_d : [a_b_c_d[0],a_b_c_d[1],a_b_c_d[2],a_b_c_d[3]])
print(Statistics.corr(data))

[[ 1.          1.         -1.          0.13172517]
 [ 1.          1.         -1.          0.13172517]
 [-1.         -1.          1.         -0.13172517]
 [ 0.13172517  0.13172517 -0.13172517  1.        ]]
